In [ ]:
#imports
import os
import pandas as pd
import re
import requests
from bs4 import BeautifulSoup
import time
from selenium import webdriver #Webdriver de Selenium qui permet de contrôler un navigateur
from selenium.webdriver.common.by import By #Permet d'accéder aux différents élements de la page web
from webdriver_manager.chrome import ChromeDriverManager #Assure la gestion du webdriver de Chrome
from datetime import datetime, timedelta
import math
import time
import warnings
warnings.filterwarnings("ignore")
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import itertools

SELENIUM

In [ ]:
options = Options()
options.headless = True
options.add_argument('--disable-blink-features=AutomationControlled')
options.add_argument("--window-size=1920,1200")
options.add_argument("--download.default_directory=E:\\Corpus\\CLAPI_scrape")

In [ ]:
driver = webdriver.Chrome(ChromeDriverManager().install(),options=options)  #Initialisation du driver
driver.implicitly_wait(0.5)
time.sleep(3) #Ajout d'un temps de deux secondes avant de lancer l'action suivante
driver.get("http://clapi.icar.cnrs.fr/V3_Feuilleter.php?")

In [ ]:
def next_corpus(driver):
    corpus_suivant = driver.find_element(By.XPATH, "//input[@value='Corpus suivant']")
    corpus_suivant.click()
    return driver

In [ ]:
def get_corpus_name(driver):
    page_source = driver.page_source
    soup = BeautifulSoup(page_source,features='html.parser')
    pattern = r'Corpus [0-9]+ sur 69 : (.*)' # Define the pattern using a regular expression
    tag = soup.find(text=re.compile(pattern)) # Find the tag that contains the pattern
    match = re.search(pattern, tag) # Extract the matching text using the regular expression pattern
    corpus_name = match.group(1) # Extract the text group and save it in the corpus_name variable
    return corpus_name # Print the corpus_name variable

In [ ]:
def get_metadata_corpus(page_html, driver):
    soup = BeautifulSoup(page_html,features='html.parser')
    pattern = r'Corpus [0-9]+ sur 69 : (.*)' 
    tag = soup.find(text=re.compile(pattern))
    match = re.search(pattern, tag) 
    corpus_name = match.group(1) 
    metadata_corpus = soup.find('div', {'id':'u262container'})
    autrice_corpus = metadata_corpus.find('div', {'id':'u274_rtf'}).get_text()
    date_collecte_corpus = metadata_corpus.find('div', {'id':'u276_rtf'}).get_text()
    duree_corpus = metadata_corpus.find('div', {'id':'u280_rtf'}).get_text()
    langue_corpus = metadata_corpus.find('div', {'id':'u282_rtf'}).get_text()
    print('Corpus_name : ', corpus_name)
    print('\n')
    print('auteur.rice : ', autrice_corpus)
    print('date_collecte_corpus : ', date_collecte_corpus)
    print('duree_corpus : ', duree_corpus)
    print('langue_corpus : ', langue_corpus)
    nb_enregistrement, liste_df = enregistrement(soup, driver)
    print('nb_enregistrement: ', nb_enregistrement)
    for df in liste_df:
        df['nom_corpus'] = corpus_name
        df['responsable'] = autrice_corpus
        df['duree_corpus'] = duree_corpus
        df['langue_corpus'] = langue_corpus
    print('___________________________________')
    return liste_df

In [ ]:
def get_element(liste):
    if not liste:
        liste = ''
    elif len(liste)>1:
        liste = liste[-1].get_text()
    elif len(liste) == 1:
        liste = liste[0].get_text()
    else : 
        liste = ''    
    return liste

In [ ]:
def enregistrement(soup, driver):
    nb_enregistrements = int(soup.find('div', {'id':'u286_rtf'}).get_text().replace(' Enregistrements ', '').replace(' Enregistrement ', ''))
    liste_enregistrement = soup.find_all('div', {'id':'ou344'})
    liste_df = []
    for i in range(nb_enregistrements):
        page_source = driver.page_source
        soup = BeautifulSoup(page_source,features='html.parser')
        nom_enregistrement = soup.find('div', {'id':'u524_rtf'}).get_text()
    
        description = soup.find_all('div', {'id':'u380'})
        description = get_element(description)
        
        data_collecte = soup.find_all('div', {'id':'u372_rtf'})
        data_collecte = get_element(data_collecte)

        duree = soup.find_all('div', {'id':'u374_rtf'})
        duree = get_element(duree)

        nb_locuteurs = soup.find_all('div', {'id':'u376_rtf'})
        nb_locuteurs = get_element(nb_locuteurs)
        
        langue_lieu = soup.find_all('div', {'id':'u378_rtf'})
        langue_lieu = get_element(langue_lieu)

        table_locuteur = soup.find_all('div', {'id':'u392container'})[-1]
        noms_locuteurs = [x.get_text() for x in table_locuteur.find_all('div', {'id':'u404_rtf'})]
        sexe_locuteurs = [x.get_text() for x in table_locuteur.find_all('div', {'id':'u406_rtf'})]
        age_locuteurs = [x.get_text() for x in table_locuteur.find_all('div', {'id':'u408_rtf'})]
        langue_locuteurs = [x.get_text() for x in table_locuteur.find_all('div', {'id':'u410_rtf'})]
        profession_locuteurs = [x.get_text() for x in table_locuteur.find_all('div', {'id':'u412_rtf'})]
        print('\n')
        print('ENREGISTREMENT {}'.format(i+1))
        print('nom_enregistrement : ', nom_enregistrement)
        print('description : ', description)
        print('data_collecte : ', data_collecte)
        print('duree : ', duree)
        print('nb_locuteurs : ', nb_locuteurs)
        print('langue_lieu : ', langue_lieu)
        print('noms_locuteurs : ', noms_locuteurs)
        print('sexe_locuteurs : ', sexe_locuteurs)
        print('age_locuteurs : ', age_locuteurs)
        print('langue_locuteurs : ', langue_locuteurs)
        print('profession_locuteurs : ', profession_locuteurs)
        dict_loc = {}
        for index in range(len(noms_locuteurs)):
            dict_loc[index] = {
        'nom': noms_locuteurs[index],
        'sexe': sexe_locuteurs[index],
        'age': age_locuteurs[index],
        'profession': profession_locuteurs[index],
        'langue': langue_locuteurs[index]}

        # Find the element using its XPath
        element = driver.find_element(By.XPATH, xpath_soup(liste_enregistrement[i]))
        driver.execute_script("arguments[0].click();", element)
        time.sleep(2)

        df = pd.DataFrame({'nom_enregistrement': nom_enregistrement,
            'description': description,
            'date_collecte': data_collecte,
            'duree': duree,
            'nb_locuteurs': nb_locuteurs,
            'langue_lieu': langue_lieu,
            'locuteurs': str(dict_loc) }, index=[0])

        liste_df.append(df)
    return nb_enregistrements, liste_df

In [ ]:
def xpath_soup(element):
    components = []
    child = element if element.name else element.parent
    for parent in child.parents:
        siblings = parent.find_all(child.name, recursive=False)
        components.append(
            child.name
            if siblings == [child] else
            '%s[%d]' % (child.name, 1 + siblings.index(child))
            )
        child = parent
    components.reverse()
    return '/%s' % '/'.join(components)

In [ ]:
liste_data_clapi = []
nom_ancien = ''
while nom_ancien != get_corpus_name(driver):
    print('nom ancien: ', nom_ancien)
    print('nom actuel: ', get_corpus_name(driver))
    liste_df = get_metadata_corpus(driver.page_source, driver)
    liste_data_clapi.append(liste_df)
    time.sleep(1)
    nom_ancien = get_corpus_name(driver)
    time.sleep(1)
    next_corpus(driver)

In [ ]:
df= liste_data_clapi[3][0]

In [ ]:
driver.quit()